In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

Using TensorFlow backend.


In [2]:
data=pd.read_csv('/Users/gauravtyagi/Downloads/data/LANL-Earthquake-Prediction/train.csv',nrows=50000)


In [3]:
dataset = data.values
dataset = data.astype('float32')

In [5]:
# normalize the dataset
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

In [6]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

33500 16500


In [7]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [8]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [9]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [10]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
 - 102s - loss: 0.0010
Epoch 2/100
 - 99s - loss: 3.5981e-04
Epoch 3/100
 - 100s - loss: 3.5964e-04
Epoch 4/100
 - 98s - loss: 3.5396e-04
Epoch 5/100
 - 100s - loss: 3.5337e-04
Epoch 6/100
 - 99s - loss: 3.5298e-04
Epoch 7/100
 - 99s - loss: 3.5268e-04
Epoch 8/100
 - 99s - loss: 3.5161e-04
Epoch 9/100
 - 99s - loss: 3.5172e-04
Epoch 10/100
 - 99s - loss: 3.4933e-04
Epoch 11/100
 - 99s - loss: 3.4816e-04
Epoch 12/100
 - 99s - loss: 3.4947e-04
Epoch 13/100
 - 99s - loss: 3.4956e-04
Epoch 14/100
 - 99s - loss: 3.4639e-04
Epoch 15/100
 - 99s - loss: 3.4623e-04
Epoch 16/100
 - 99s - loss: 3.4571e-04
Epoch 17/100
 - 99s - loss: 3.4478e-04
Epoch 18/100
 - 100s - loss: 3.4443e-04
Epoch 19/100
 - 101s - loss: 3.4326e-04
Epoch 20/100
 - 103s - loss: 3.4231e-04
Epoch 21/100
 - 101s - loss: 3.4262e-04
Epoch 22/100
 - 103s - loss: 3.4210e-04
Epoch 23/100
 - 104s - loss: 3.4231e-04
Epoch 24/100
 - 101s - loss: 3.4207e-04
Epoch 25/100
 - 103s - loss: 3.4134e-04
Epoch 26/100
 - 99s - loss:

In [11]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

NameError: name 'model' is not defined

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
#hjbkdkjaeshfkjas